# Baseline Seizure Detection Model with Performance Metrics
## Motivation
We implemented baseline models such as Logistric regression, SVM, ANN, and Inception Nucleus. Their performance was evaluated using standard metrics.

## Conclusion
The models already demonstrate strong performance in seizure detection, achieving an accuracy of over 97%. To further evaluate the models' robustness, we plan to generate challenging test cases where seizure detection is more difficult. This will help clearly identify the differences in performance before and after applying data augmentation.

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import StandardScaler

In [18]:
# Load cleaned data
df = pd.read_csv('data/data_cleaned.csv')
X = df.loc[:, 'X1':'X178']  # All feature columns
y = df['y']  # Labels (non-seizure/seizure)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### 1. Logistic Regression
#### 1.1 Imbalanced dataset

In [50]:

def evaluate_classification_metrics(y_true, y_pred, y_pred_prob):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate precision
    precision = precision_score(y_true, y_pred)
    
    # Calculate recall
    recall = recall_score(y_true, y_pred)
    
    # Calculate F1 score
    f1 = f1_score(y_true, y_pred)
    
    # Calculate ROC-AUC score
    roc_auc = roc_auc_score(y_true, y_pred_prob)
    
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(y_true, y_pred)
    
    # Print metrics
    print(f'Accuracy: {accuracy * 100:.2f} %')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')
    print(f'ROC-AUC: {roc_auc:.2f}')
    print(f'Cohen\'s Kappa: {kappa:.2f}')
    
    # Return metrics as a dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'kappa': kappa
    }

In [51]:
# Necassary functions

def evaluate_classification_metrics(y_ground_truth, y_pred, y_pred_prob):
    # Calculate accuracy
    accuracy = accuracy_score(y_ground_truth, y_pred)
    
    # Calculate precision
    precision = precision_score(y_ground_truth, y_pred)
    
    # Calculate recall
    recall = recall_score(y_ground_truth, y_pred)
    
    # Calculate F1 score
    f1 = f1_score(y_ground_truth, y_pred, average='weighted')
    
    if len(set(y_ground_truth)) > 1:
        # Check if ROC-AUC can be calculated (i.e., both classes are present)
        roc_auc = roc_auc_score(y_ground_truth, y_pred_prob)
        # Calculate Cohen's Kappa
        kappa = cohen_kappa_score(y_ground_truth, y_pred)
    else:
        roc_auc = None  # Not computable, only one class in y_true
        kappa = None
    
    # Calculate metrics for seizure class (y_label=1)
    precision_seizure = precision_score(y_ground_truth, y_pred, pos_label=1)
    recall_seizure = recall_score(y_ground_truth, y_pred, pos_label=1)
    f1_seizure = f1_score(y_ground_truth, y_pred, pos_label=1)
    
    # Calculate metrics for non-seizure class (y_label=0)
    precision_non_seizure = precision_score(y_ground_truth, y_pred, pos_label=0)
    recall_non_seizure = recall_score(y_ground_truth, y_pred, pos_label=0)
    f1_non_seizure = f1_score(y_ground_truth, y_pred, pos_label=0)
    
    print(f'\nSeizure (y=1):')
    print(f'  Precision: {precision_seizure * 100:.2f} %')
    print(f'  Recall: {recall_seizure * 100:.2f} %')
    print(f'  F1 Score: {f1_seizure * 100:.2f} %')
    
    print(f'\nNon-Seizure (y=0):')
    print(f'  Precision: {precision_non_seizure * 100:.2f} %')
    print(f'  Recall: {recall_non_seizure * 100:.2f} %')
    print(f'  F1 Score: {f1_non_seizure * 100:.2f} %')
    
    # Print metrics
    print(f'\nOverall:')
    print(f'  Accuracy: {accuracy * 100:.2f} %')
    print(f'  Precision: {precision * 100:.2f} %')
    print(f'  Recall: {recall * 100:.2f} %')
    print(f'  F1 Score: {f1 * 100:.2f} %')
    if roc_auc is not None:
        print(f'  ROC-AUC: {roc_auc * 100:.2f} %')
    if kappa is not None:
        print(f'  Cohen\'s Kappa: {kappa * 100:.2f} %')


    return {
        'Seizure Precision': precision_seizure,
        'Seizure Recall': recall_seizure,
        'Seizure F1 Score': f1_seizure,
        'Non-Seizure Precision': precision_non_seizure,
        'Non-Seizure Recall': recall_non_seizure,
        'Non-Seizure F1 Score': f1_non_seizure,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC_AUC': roc_auc,
        "Cohen's Kappa": kappa,
    }

In [52]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(max_iter=200)
model_lr.fit(X_train, y_train)
y_pred_log_reg = model_lr.predict(X_test)
# acc_log_reg = round(clf.score(X_train, y_train) * 100, 2)
y_pred_prob = model_lr.predict_proba(X_test)[:, 1]  # Probability estimates for ROC-AUC
metrics_lr = evaluate_classification_metrics(y_test, y_pred_log_reg, y_pred_prob)


Seizure (y=1):
  Precision: 97.96 %
  Recall: 10.32 %
  F1 Score: 18.68 %

Non-Seizure (y=0):
  Precision: 81.47 %
  Recall: 99.95 %
  F1 Score: 89.77 %

Overall:
  Accuracy: 81.83 %
  Precision: 97.96 %
  Recall: 10.32 %
  F1 Score: 75.40 %
  ROC-AUC: 54.04 %
  Cohen's Kappa: 15.42 %


#### Observations:
1. The ROC-AUC score of 0.55 is very close to 0.5, which suggests that the model’s ability to distinguish between the positive and negative classes is almost random.
2. A recall of 0.28 means that the model identifies only 28% of the actual positive cases. This suggests that the model is missing many actual positives (high false negatives).

#### 1.2  Balanced dataset

In [53]:
from imblearn.over_sampling import SMOTE
# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [54]:
from sklearn.linear_model import LogisticRegression
model_lr_bal = LogisticRegression(max_iter=200)
model_lr_bal.fit(X_train_balanced, y_train_balanced)
y_pred_log_reg = model_lr_bal.predict(X_test)
# acc_log_reg = round(clf.score(X_train, y_train) * 100, 2)
y_pred_prob = model_lr_bal.predict_proba(X_test)[:, 1]  # Probability estimates for ROC-AUC
metrics_lr_bal = evaluate_classification_metrics(y_test, y_pred_log_reg, y_pred_prob)


Seizure (y=1):
  Precision: 33.65 %
  Recall: 46.02 %
  F1 Score: 38.87 %

Non-Seizure (y=0):
  Precision: 84.92 %
  Recall: 77.00 %
  F1 Score: 80.77 %

Overall:
  Accuracy: 70.74 %
  Precision: 33.65 %
  Recall: 46.02 %
  F1 Score: 72.30 %
  ROC-AUC: 53.83 %
  Cohen's Kappa: 20.25 %


/opt/conda/envs/capstone/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 2.Support Vector Machine (SVM)


#### 2.1 Balanced dataset

In [55]:
from sklearn.svm import SVC
model_svc = SVC()
model_svc.fit(X_train, y_train)
y_pred_svc = model_svc.predict(X_test)
y_pred_prob = model_svc.decision_function(X_test) 
metrics_svc = evaluate_classification_metrics(y_test, y_pred_svc, y_pred_prob)


Seizure (y=1):
  Precision: 96.84 %
  Recall: 92.26 %
  F1 Score: 94.49 %

Non-Seizure (y=0):
  Precision: 98.06 %
  Recall: 99.24 %
  F1 Score: 98.65 %

Overall:
  Accuracy: 97.83 %
  Precision: 96.84 %
  Recall: 92.26 %
  F1 Score: 97.81 %
  ROC-AUC: 99.70 %
  Cohen's Kappa: 93.14 %


#### 2.2 Imbalanced dataset

In [56]:
# Train SVC with balanced data
model_svc_bal = SVC()
model_svc_bal.fit(X_train_balanced, y_train_balanced)

# Make predictions on original test set
y_pred_svc = model_svc_bal.predict(X_test)
y_pred_prob = model_svc_bal.decision_function(X_test)

# Evaluate metrics
metrics_svc_bal = evaluate_classification_metrics(y_test, y_pred_svc, y_pred_prob)


Seizure (y=1):
  Precision: 92.99 %
  Recall: 96.99 %
  F1 Score: 94.95 %

Non-Seizure (y=0):
  Precision: 99.23 %
  Recall: 98.15 %
  F1 Score: 98.68 %

Overall:
  Accuracy: 97.91 %
  Precision: 92.99 %
  Recall: 96.99 %
  F1 Score: 97.93 %
  ROC-AUC: 99.57 %
  Cohen's Kappa: 93.63 %


This shows significant improvement in performance

### 3. Linear SVM

#### 3.1 Balanced dataset

In [57]:
from sklearn.svm import SVC, LinearSVC
model_lsvc = LinearSVC()
model_lsvc.fit(X_train, y_train)
y_pred_linear_svc = model_lsvc.predict(X_test)
y_pred_prob = model_lsvc.decision_function(X_test)
metrics_l_svc = evaluate_classification_metrics(y_test, y_pred_linear_svc, y_pred_prob)


Seizure (y=1):
  Precision: 92.86 %
  Recall: 8.39 %
  F1 Score: 15.38 %

Non-Seizure (y=0):
  Precision: 81.13 %
  Recall: 99.84 %
  F1 Score: 89.52 %

Overall:
  Accuracy: 81.35 %
  Precision: 92.86 %
  Recall: 8.39 %
  F1 Score: 74.53 %
  ROC-AUC: 53.86 %
  Cohen's Kappa: 12.45 %


In [58]:
from sklearn.svm import SVC, LinearSVC
model_l_svc_bal = LinearSVC()
model_l_svc_bal.fit(X_train_balanced, y_train_balanced)
y_pred_linear_svc = model_l_svc_bal.predict(X_test)
y_pred_prob = model_l_svc_bal.decision_function(X_test)
metrics_l_svc_bal = evaluate_classification_metrics(y_test, y_pred_linear_svc, y_pred_prob)


Seizure (y=1):
  Precision: 32.77 %
  Recall: 45.59 %
  F1 Score: 38.13 %

Non-Seizure (y=0):
  Precision: 84.69 %
  Recall: 76.29 %
  F1 Score: 80.28 %

Overall:
  Accuracy: 70.09 %
  Precision: 32.77 %
  Recall: 45.59 %
  F1 Score: 71.75 %
  ROC-AUC: 53.73 %
  Cohen's Kappa: 19.10 %


### 4. *k*-Nearest Neighbors


#### 4.1 Imbalanced dataset

In [59]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)
y_pred_prob = model_knn.predict_proba(X_test)[:, 1]
metrics_knn = evaluate_classification_metrics(y_test, y_pred_knn, y_pred_prob)


Seizure (y=1):
  Precision: 99.33 %
  Recall: 63.44 %
  F1 Score: 77.43 %

Non-Seizure (y=0):
  Precision: 91.51 %
  Recall: 99.89 %
  F1 Score: 95.52 %

Overall:
  Accuracy: 92.52 %
  Precision: 99.33 %
  Recall: 63.44 %
  F1 Score: 91.86 %
  ROC-AUC: 92.36 %
  Cohen's Kappa: 73.20 %


#### 4.2 Balanced dataset

In [60]:
from sklearn.neighbors import KNeighborsClassifier
model_knn_bal = KNeighborsClassifier()
model_knn_bal.fit(X_train_balanced, y_train_balanced)
y_pred_knn = model_knn_bal.predict(X_test)
y_pred_prob = model_knn_bal.predict_proba(X_test)[:, 1]
metrics_knn_bal = evaluate_classification_metrics(y_test, y_pred_knn, y_pred_prob)


Seizure (y=1):
  Precision: 97.54 %
  Recall: 85.38 %
  F1 Score: 91.06 %

Non-Seizure (y=0):
  Precision: 96.41 %
  Recall: 99.46 %
  F1 Score: 97.91 %

Overall:
  Accuracy: 96.61 %
  Precision: 97.54 %
  Recall: 85.38 %
  F1 Score: 96.52 %
  ROC-AUC: 95.12 %
  Cohen's Kappa: 88.97 %


### 5. Gaussian Naive Bayes


#### 5.1 Imbalanced dataset

In [61]:
from sklearn.naive_bayes import GaussianNB
model_gnb = GaussianNB()
model_gnb.fit(X_train, y_train)
y_pred_gnb = model_gnb.predict(X_test)
y_pred_prob = model_gnb.predict_proba(X_test)[:, 1]
metrics_gnb = evaluate_classification_metrics(y_test, y_pred_gnb, y_pred_prob)


Seizure (y=1):
  Precision: 90.99 %
  Recall: 91.18 %
  F1 Score: 91.08 %

Non-Seizure (y=0):
  Precision: 97.76 %
  Recall: 97.71 %
  F1 Score: 97.74 %

Overall:
  Accuracy: 96.39 %
  Precision: 90.99 %
  Recall: 91.18 %
  F1 Score: 96.39 %
  ROC-AUC: 98.59 %
  Cohen's Kappa: 88.82 %


#### 5.2 Balanced dataset

In [ ]:
from sklearn.naive_bayes import GaussianNB
model_gnb_bal = GaussianNB()
model_gnb_bal.fit(X_train_balanced, y_train_balanced)
y_pred_gnb = model_gnb_bal.predict(X_test)
y_pred_prob = model_gnb_bal.predict_proba(X_test)[:, 1]
metrics_gnb_bal = evaluate_classification_metrics(y_test, y_pred_gnb, y_pred_prob)


Seizure (y=1):
  Precision: 89.30 %
  Recall: 93.33 %
  F1 Score: 91.27 %

Non-Seizure (y=0):
  Precision: 98.29 %
  Recall: 97.17 %
  F1 Score: 97.73 %

Overall:
  Accuracy: 96.39 %
  Precision: 89.30 %
  Recall: 93.33 %
  F1 Score: 96.42 %
  ROC-AUC: 98.44 %
  Cohen's Kappa: 89.00 %


### 6. Artificial Neural Networks(ANN)


#### 6.1 Imbalanced dataset

In [63]:
#Importing keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

model_ann = Sequential()
model_ann.add(Dense(units=80, kernel_initializer='uniform', activation='relu', input_dim=178))
model_ann.add(Dense(units=80, kernel_initializer='uniform', activation='relu',))
model_ann.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model_ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting the ANN to the training set
model_ann.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
920/920 [==============================] - 3s 2ms/step - loss: 0.2082 - accuracy: 0.9451
Epoch 2/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0922 - accuracy: 0.9720
Epoch 3/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0829 - accuracy: 0.9735
Epoch 4/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0665 - accuracy: 0.9770
Epoch 5/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0590 - accuracy: 0.9791
Epoch 6/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0538 - accuracy: 0.9809
Epoch 7/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0526 - accuracy: 0.9811
Epoch 8/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0423 - accuracy: 0.9860
Epoch 9/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0403 - accuracy: 0.9845
Epoch 10/100
920/920 [==============================] - 2s 2ms/step - loss: 0.0376 - accura

In [64]:
# Predicting the Test set results
y_pred_probs = model_ann.predict(X_test)
y_pred = (y_pred_probs > 0.5)
metrics_ann = evaluate_classification_metrics(y_test, y_pred, y_pred_probs)

72/72 [==============================] - 0s 1ms/step

Seizure (y=1):
  Precision: 95.97 %
  Recall: 92.26 %
  F1 Score: 94.08 %

Non-Seizure (y=0):
  Precision: 98.06 %
  Recall: 99.02 %
  F1 Score: 98.54 %

Overall:
  Accuracy: 97.65 %
  Precision: 95.97 %
  Recall: 92.26 %
  F1 Score: 97.63 %
  ROC-AUC: 99.25 %
  Cohen's Kappa: 92.62 %


#### 6.2 Balanced dataset

In [65]:
model_ann_bal = Sequential()
model_ann_bal.add(Dense(units=80, kernel_initializer='uniform', activation='relu', input_dim=178))
model_ann_bal.add(Dense(units=80, kernel_initializer='uniform', activation='relu',))
model_ann_bal.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model_ann_bal.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting the ANN to the training set
model_ann_bal.fit(X_train_balanced, y_train_balanced, batch_size = 10, epochs = 100)

Epoch 1/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.2029 - accuracy: 0.9290
Epoch 2/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.1020 - accuracy: 0.9679
Epoch 3/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0795 - accuracy: 0.9745
Epoch 4/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0659 - accuracy: 0.9768
Epoch 5/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0576 - accuracy: 0.9791
Epoch 6/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0450 - accuracy: 0.9840
Epoch 7/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0397 - accuracy: 0.9855
Epoch 8/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0301 - accuracy: 0.9887
Epoch 9/100
1473/1473 [==============================] - 3s 2ms/step - loss: 0.0305 - accuracy: 0.9889
Epoch 10/100
1473/1473 [==============================] - 3s 2ms/step - l

In [66]:
# Predicting the Test set results
y_pred_probs = model_ann_bal.predict(X_test)
y_pred = (y_pred_probs > 0.5)
metrics_ann_bal = evaluate_classification_metrics(y_test, y_pred, y_pred_probs)

72/72 [==============================] - 0s 1ms/step

Seizure (y=1):
  Precision: 94.91 %
  Recall: 92.26 %
  F1 Score: 93.57 %

Non-Seizure (y=0):
  Precision: 98.05 %
  Recall: 98.75 %
  F1 Score: 98.40 %

Overall:
  Accuracy: 97.43 %
  Precision: 94.91 %
  Recall: 92.26 %
  F1 Score: 97.42 %
  ROC-AUC: 99.07 %
  Cohen's Kappa: 91.96 %


## Ablation study

In [68]:
metrics_data = {
    'Logistic Regression': metrics_lr,
    'SVC': metrics_svc,
    'KNN': metrics_knn,
    'Naive Bayes': metrics_gnb,
    'Linear SVC': metrics_l_svc,
    'ANN': metrics_ann,
}

df_metrics = pd.DataFrame(metrics_data)
df_metrics.T.sort_values(by='ROC_AUC', ascending=False)

,Seizure Precision,Seizure Recall,Seizure F1 Score,Non-Seizure Precision,Non-Seizure Recall,Non-Seizure F1 Score,Accuracy,Precision,Recall,F1 Score,ROC_AUC,Cohen's Kappa
SVC,0.968397,0.922581,0.944934,0.980614,0.992371,0.986457,0.978261,0.968397,0.922581,0.978062,0.997047,0.931401
ANN,0.959732,0.922581,0.940789,0.980572,0.990191,0.985358,0.976522,0.959732,0.922581,0.976347,0.992532,0.926155
Naive Bayes,0.909871,0.911828,0.910849,0.977644,0.977112,0.977378,0.963913,0.909871,0.911828,0.963928,0.985880,0.888227
KNN,0.993266,0.634409,0.774278,0.915127,0.998910,0.955185,0.925217,0.993266,0.634409,0.918610,0.923589,0.732049
Logistic Regression,0.979592,0.103226,0.186770,0.814749,0.999455,0.897699,0.818261,0.979592,0.103226,0.753968,0.540355,0.154166
Linear SVC,0.928571,0.083871,0.153846,0.811337,0.998365,0.895187,0.813478,0.928571,0.083871,0.745307,0.538601,0.124521


In [70]:
metrics_data = {
    'Logistic Regression': metrics_lr_bal,
    'SVC': metrics_svc_bal,
    'KNN': metrics_knn_bal,
    'Naive Bayes': metrics_gnb_bal,
    'Linear SVC': metrics_l_svc_bal,
    'ANN': metrics_ann_bal,
}

df_metrics = pd.DataFrame(metrics_data)
df_metrics.T.sort_values(by='ROC_AUC', ascending=False)


,Seizure Precision,Seizure Recall,Seizure F1 Score,Non-Seizure Precision,Non-Seizure Recall,Non-Seizure F1 Score,Accuracy,Precision,Recall,F1 Score,ROC_AUC,Cohen's Kappa
SVC,0.929897,0.969892,0.949474,0.992287,0.981471,0.986849,0.979130,0.929897,0.969892,0.979293,0.995675,0.936330
ANN,0.949115,0.922581,0.935660,0.980519,0.987466,0.983980,0.974348,0.949115,0.922581,0.974211,0.990744,0.919644
Naive Bayes,0.893004,0.933333,0.912723,0.982911,0.971662,0.977254,0.963913,0.893004,0.933333,0.964208,0.984420,0.889991
KNN,0.975430,0.853763,0.910550,0.964078,0.994550,0.979077,0.966087,0.975430,0.853763,0.965223,0.951248,0.889742
Logistic Regression,0.336478,0.460215,0.388738,0.849159,0.770027,0.807659,0.707391,0.336478,0.460215,0.722964,0.538294,0.202451
Linear SVC,0.327666,0.455914,0.381295,0.846945,0.762943,0.802752,0.700870,0.327666,0.455914,0.717545,0.537314,0.190957


## Emsembling 
Since SVC, ANN, and Naive Bayes have the best performance, we will emsemble them to check if the performance can be further enhanced.

In [87]:
from scipy import stats
from scipy.special import expit  # For sigmoid conversion

# Predictions and probabilities from the models
pred1 = model_svc_bal.predict(X_test)
pred2 = model_gnb_bal.predict(X_test)
probs3 = model_ann.predict(X_test).squeeze()  # ANN output probabilities

# Convert decision function to probabilities for SVM
probs1 = expit(model_svc_bal.decision_function(X_test))
probs2 = model_gnb_bal.predict_proba(X_test)[:, 1]  # Get probabilities for class 1

# Average the predicted probabilities for each instance
final_probs = np.mean(np.stack([probs1, probs2, probs3]), axis=0)

# Ensure ANN predictions are in the same format as pred1 and pred2
pred3 = (probs3 > 0.5).astype(int).flatten()

# Print shapes for debugging
print("Shape of predictions:")
print(f"pred1: {pred1.shape}, pred2: {pred2.shape}, pred3: {pred3.shape}")

# Majority voting ensemble
final_predictions = stats.mode(np.stack([pred1, pred2, pred3]), axis=0)[0].flatten()

# Ensure ROC-AUC is calculated with proper probabilities
if len(set(y_test)) > 2:  # Multiclass case
    metrics = evaluate_classification_metrics(y_test, final_predictions, final_probs, multi_class='ovr')
else:  # Binary case
    metrics = evaluate_classification_metrics(y_test, final_predictions, final_probs)


72/72 [==============================] - 0s 1ms/step
Shape of predictions:
pred1: (2300,), pred2: (2300,), pred3: (2300,)

Seizure (y=1):
  Precision: 93.71 %
  Recall: 96.13 %
  F1 Score: 94.90 %

Non-Seizure (y=0):
  Precision: 99.01 %
  Recall: 98.37 %
  F1 Score: 98.69 %

Overall:
  Accuracy: 97.91 %
  Precision: 93.71 %
  Recall: 96.13 %
  F1 Score: 97.92 %
  ROC-AUC: 99.66 %
  Cohen's Kappa: 93.59 %


### Conclusion:

1. SVC, ANN, Naive Bayes have the best performance, with 0.978062	0.997047	0.931401
2. The model has already achieved amazing performance, emsembling models can barely improve more.